# Efficient Large Language Model Inference with SpecKV-PC
This notebook demonstrates how to patch a pre-trained LLM with SpecKV-PC. We'll also evaluate with a synthetic retrieval-style prompt.

In [1]:
import os
import random
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# (Optional) Ensure only the first GPU is used (if available)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Ensure deterministic shuffling
random.seed(42)

from draft_approx_llm import SpecKVConfig, SpecPCConfig, SpecKVPCConfig, patch_model

/home/kevin/anaconda3/envs/draftapprox/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Helper Function for Prompt Generation

In [2]:
def create_niah_example(n_repeat: int = 100, key: str = "123456789") -> str:
    """
    Generates a synthetic document with many distractors and a hidden key to test retrieval abilities.
    """
    example_sentences = [
        "The cat sat on the mat.",
        "The quick brown fox jumps over the lazy dog.",
        "A journey of a thousand miles begins with a single step.",
        "To be or not to be, that is the question.",
        "All that glitters is not gold."
    ]

    context = []
    for sentence in example_sentences:
        context.extend([sentence] * n_repeat)
    context.append(f"The key: {key}")
    random.Random(42).shuffle(context)  # deterministic shuffle
    
    prompt = "\n".join(context)
    prompt += "\n\nWhat is the key?"
    return prompt

## Model and Tokenizer Loading

In [3]:
# Model configurations: adjust as needed
MAIN_MODEL_NAME = "Qwen/Qwen2.5-14B-Instruct"
DRAFT_MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"

model_kwargs = {
    "torch_dtype": torch.bfloat16,
    "attn_implementation": "flash_attention_2",
    "device_map": "auto"
}

# Load models and tokenizer
tokenizer = AutoTokenizer.from_pretrained(MAIN_MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MAIN_MODEL_NAME, **model_kwargs)
draft_model = AutoModelForCausalLM.from_pretrained(DRAFT_MODEL_NAME, **model_kwargs)

Loading checkpoint shards: 100%|██████████| 8/8 [00:03<00:00,  2.31it/s]
Some parameters are on the meta device because they were offloaded to the cpu.


## SpecKV-PC Configuration and Patching

In [ ]:
# Adjust these hyperparameters to tune performance/quality tradeoff
sparse_config = SpecKVPCConfig(
    specpc_config=SpecPCConfig(
        max_capacity_prompt=2048,
        window_size=64,
        pool_type="max",
        kernel_size=64,
        reduction_type="max",
        lookahead_tokens=8,
        neighbor_tokens=64,
        starting_layer_index=8,
        weighted_query=True
    ),
    speckv_config=SpecKVConfig(
        max_capacity_prompt=256,
        window_size=32,
        pool_type="max",
        kernel_size=7,
        reduction_type="max",
        prefill_window_size=2048,
        prefill_vertical_size=2048
    )
)

# Patch main model for SpecKV
patched_model = patch_model(model, draft_model, sparse_config)

## Prepare Prompt and Tokenized Input

In [5]:
sample_input = create_niah_example()
inputs = tokenizer(sample_input, return_tensors="pt").to(patched_model.device)
print(f"Input token count: {inputs.input_ids.shape[1]}")

Input token count: 4919


## Generate Output

In [6]:
with torch.inference_mode():
    gen_outputs = patched_model.generate(
        **inputs,
        max_new_tokens=32,
        do_sample=False,
        temperature=None,
        top_p=None,
        top_k=None,
        return_dict_in_generate=True
    )

To benefit from fast kernel implementations, we recommend installing SGLang or vllm.


## Decode and Display Model Output

In [ ]:
decoded_output = tokenizer.decode(gen_outputs.output_ids[0], skip_special_tokens=True).strip()
print(decoded_output)

The key provided in the text is "123456789". However, it's important to note that this key does not seem to


: 